In [4]:
import os
import random
import shutil
from pathlib import Path

# Paths
source_images = Path(r"C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\full_dataset\images")
source_labels = Path(r"C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\full_dataset\labels")
dest_base = Path("balanced_dataset_split_2")

# Destination folders
train_images = dest_base / "train_80/images"
train_labels = dest_base / "train_80/labels"
fl_images = dest_base / "train_20/images"
fl_labels = dest_base / "train_20/labels"

# Make sure all destination folders exist
for folder in [train_images, train_labels, fl_images, fl_labels]:
    folder.mkdir(parents=True, exist_ok=True)

# List all images (assuming .jpg or .png)
image_files = sorted(list(source_images.glob("*.jpg")) + list(source_images.glob("*.png")))
random.shuffle(image_files)

# Split
split_idx = int(0.95 * len(image_files))
train_set = image_files[:split_idx]
fl_set = image_files[split_idx:]

def move_files(image_list, target_img_dir, target_lbl_dir):
    for img_path in image_list:
        label_path = source_labels / (img_path.stem + ".txt")
        if label_path.exists():
            shutil.copy2(img_path, target_img_dir / img_path.name)
            shutil.copy2(label_path, target_lbl_dir / label_path.name)

# Move the files
move_files(train_set, train_images, train_labels)
move_files(fl_set, fl_images, fl_labels)

print(f"Done. {len(train_set)} images in train_80, {len(fl_set)} in train_20.")


Done. 6118 images in train_80, 323 in train_20.


In [ ]:
from ultralytics import YOLO
from clearml import Task
import random
import shutil
from pathlib import Path
import pandas as pd
from clearml import Task as ClearMLTask
import os

# 🟦 Main ClearML parent task
parent_task = Task.init(
    project_name="YOLOv9t Tuning Extended_2",
    task_name="YOLOv9t Inline HPO (with Baselines)",
    task_type=Task.TaskTypes.optimizer
)

dataset_yaml = "balanced_dataset_split/train_80.yaml"
results_path = Path("runs/detect")
run_metrics = []

# 🎯 Training function with ClearML logging
def train_model(hyperparameters, label="HPO-Random"):
    run_name = f"yolo9t_{label}_lr{hyperparameters['lr0']}_mom{hyperparameters['momentum']}_aug{hyperparameters['mosaic']}"

    task = Task.create(
        project_name="YOLOv9t Tuning Extended_2",
        task_name=f"YOLOv9t Run - {run_name}",
        task_type=Task.TaskTypes.training
    )
    task.connect(hyperparameters)
    task.add_tags(["YOLOv9t-HPO", label])
    Task.current_task().close()  # ❗️Close previous active task context
    Task._Task__default_task = task  # 🧠 HACK: switch the active logging context
    model = YOLO("yolov9t.pt")
    results = model.train(
        optimizer="SGD",
        data=dataset_yaml,
        imgsz=256,
        epochs=50,
        batch=180,
        lr0=hyperparameters["lr0"],
        momentum = max(0.01, hyperparameters["momentum"]),  # Clamp to avoid PyTorch crash
        weight_decay=hyperparameters["weight_decay"],
        warmup_epochs=int(hyperparameters["warmup_epochs"]),
        hsv_h=hyperparameters["hsv_h"],
        mosaic=hyperparameters["mosaic"],
        mixup=hyperparameters["mixup"],
        degrees=hyperparameters["degrees"],
        scale=hyperparameters["scale"],
        project="clearml_yolo9_tuning_extended",
        name=run_name,
        save=True,
        verbose=True,
    )

    try:
        map_5095 = results.box.map
        print(f"✅ {label} → Detected mAP50-95: {map_5095:.4f}")
        task.get_logger().report_scalar(
            title="metrics",
            series="mAP50-95",
            value=map_5095,
            iteration=0
        )
    except Exception as e:
        print(f"⚠️ {label} → Failed to extract mAP50-95:", e)
        map_5095 = 0.0

    if hasattr(results.box, "ap_class") and results.box.ap_class is not None:
        class_ap = results.box.ap_class()
        class_names = results.names
        for cls_id, ap in enumerate(class_ap):
            task.get_logger().report_scalar(
                title="Class-wise AP",
                series=f"{class_names[cls_id]}",
                value=ap,
                iteration=results.epoch
            )

    run_metrics.append((map_5095, results.save_dir, label, task.id))
    task.close()
    return map_5095

# 🔁 Search space (no cls/obj)
search_space = {
    "lr0": [1e-3, 5e-4],
    "momentum": [0.85, 0.90],
    "weight_decay": [1e-5, 5e-4],
    "warmup_epochs": [1, 3],
    "hsv_h": [0.0, 0.05],
    "mosaic": [0.7, 1.0],
    "mixup": [0.0, 0.2],
    "degrees": [0, 10],
    "scale": [0.5, 1.0],
}

# 🔴 
baseline = {
    "lr0": 0.01,              # Slightly too high
    "momentum": 0.6,          # Low, less stable updates
    "weight_decay": 1e-2,     # Higher than normal, may over-regularize
    "warmup_epochs": 0,       # No warmup can cause bad initial updates
    "hsv_h": 0.2,             # Over-colored images
    "mosaic": 0.2,            # Little spatial augmentation
    "mixup": 0.6,             # Too much blending
    "degrees": 25,            # Too much rotation
    "scale": 0.4              # Reduced object scale variance
}


train_model(baseline, label="Baseline-v1")

# 🔁 Run 3 randomized trials
for _ in range(3):
    hparams = {k: random.choice(v) for k, v in search_space.items()}
    train_model(hparams, label="HPO-Random")

# 📦 Save best/worst models
run_metrics.sort(key=lambda x: x[0], reverse=True)
(best_score, best_dir, best_label, best_id), (worst_score, worst_dir, worst_label, worst_id) = run_metrics[0], run_metrics[-1]

save_root = Path("saved_models")
save_root.mkdir(exist_ok=True)

shutil.copy(best_dir / "weights/best.pt", save_root / "best.pt")
with open(save_root / "best_metrics.txt", "w") as f:
    f.write(f"Best mAP50-95: {best_score:.4f}\\nFrom: {best_dir.name} ({best_label})\\n")

shutil.copy(worst_dir / "weights/best.pt", save_root / "worst.pt")
with open(save_root / "worst_metrics.txt", "w") as f:
    f.write(f"Worst mAP50-95: {worst_score:.4f}\\nFrom: {worst_dir.name} ({worst_label})\\n")

# 🏷 Tag in ClearML
ClearMLTask.get_task(task_id=best_id).add_tags(["Best-Model"])
ClearMLTask.get_task(task_id=worst_id).add_tags(["Worst-Model"])
    
# 📊 Export to CSV
df = pd.DataFrame(run_metrics, columns=["mAP50-95", "run_dir", "label", "task_id"])
df.sort_values("mAP50-95", ascending=False).to_csv("saved_models/hpo_summary.csv", index=False)

print(f"\n✅ Best: {best_label} → {best_score:.4f}")
print(f"❌ Worst: {worst_label} → {worst_score:.4f}")
print(f"📊 Absolute Gain: {(best_score - worst_score):.4f}")
print(f"📈 Relative % Gain: {100 * (best_score - worst_score) / (worst_score + 1e-9):.2f}%")


ClearML Task: created new task id=6d448df589a349b89d3ef64a4323afa8
2025-05-09 13:33:20,344 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/e24e8d821f1f41caa565498714e9583a/experiments/6d448df589a349b89d3ef64a4323afa8/output/log
New https://pypi.org/project/ultralytics/8.3.130 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.114  Python-3.9.18 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov9t.pt, data=balanced_dataset_split/train_80.yaml, epochs=50, time=None, patience=100, batch=180, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=clearml_yolo9_tuning_extended, name=yolo9t_HPO-Random_lr0.0005_mom0.85_aug1.0, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=

train: Scanning C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\labels.cache... 29477 images, 2570 backgrounds, 0 corrupt: 100%|██████████| 29477/29477 [00:00<?, ?it/s]

train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478021875081281646_jpg.rf.bEZPhuyXU5hIovwQSTIp.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478021875081281646_jpg.rf.e9552980cf8c6fef4aa02cb84c6364f5.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478897760163798179_jpg.rf.5Pzrj3Eg3vZuyl7ztKAt.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478897760163798179_jpg.rf.98623be50b02ff17d58f89fddf7a0c6c.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Se

val: Fast image access  (ping: 0.20.1 ms, read: 83.129.0 MB/s, size: 36.7 KB)


val: Scanning C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset\valid\labels.cache... 2980 images, 350 backgrounds, 0 corrupt: 100%|██████████| 2980/2980 [00:00<?, ?it/s]


...

                   all       2980      19514      0.187      0.198      0.154     0.0728

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/50      5.09G      1.704      1.294      1.157       1488        256: 100%|██████████| 164/164 [01:48<00:00,  1.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:17<00:00,  1.91s/it]


                   all       2980      19514      0.189      0.203       0.16     0.0768

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/50      4.97G      1.689      1.272      1.151       1814        256:  89%|████████▉ | 146/164 [01:45<00:13,  1.38it/s]


KeyboardInterrupt: 

In [4]:

from ultralytics import YOLO
from clearml import Task
import random
import shutil
from pathlib import Path
import pandas as pd
from clearml import Task as ClearMLTask
import os


# 🟦 Main ClearML parent task
parent_task = Task.init(
    project_name="YOLOv9t Tuning Extended_2",
    task_name="YOLOv9t Inline HPO (with Baselines)",
    task_type=Task.TaskTypes.optimizer
)


dataset_yaml = "balanced_dataset_split/train_80.yaml"
results_path = Path("runs/detect")
run_metrics = []

def train_model(hyperparameters, label="HPO-Random"):
    run_name = f"yolo9t_{label}_lr{hyperparameters['lr0']}"

    task = Task.create(
        project_name="YOLOv9t Tuning Extended_2",
        task_name=f"YOLOv9t Run - {run_name}",
        task_type=Task.TaskTypes.training
    )
    task.connect(hyperparameters)
    task.add_tags(["YOLOv9t-HPO", label])
    Task.current_task().close()  # ❗️Close previous active task context
    Task._Task__default_task = task  # 🧠 HACK: switch the active logging context
    model = YOLO("yolov9t.pt")
    results = model.train(
        optimizer="Adam",
        data=dataset_yaml,
        imgsz=256,
        epochs=50,
        batch=180,
        lr0=hyperparameters["lr0"],
        project="clearml_yolo9_tuning_extended",
        name=run_name,
        save=True,
        verbose=True,
    )

    try:
        map_5095 = results.box.map
        print(f"✅ {label} → Detected mAP50-95: {map_5095:.4f}")
        task.get_logger().report_scalar(
            title="metrics",
            series="mAP50-95",
            value=map_5095,
            iteration=0
        )
    except Exception as e:
        print(f"⚠️ {label} → Failed to extract mAP50-95:", e)
        map_5095 = 0.0

    if hasattr(results.box, "ap_class") and results.box.ap_class is not None:
        class_ap = results.box.ap_class()
        class_names = results.names
        for cls_id, ap in enumerate(class_ap):
            task.get_logger().report_scalar(
                title="Class-wise AP",
                series=f"{class_names[cls_id]}",
                value=ap,
                iteration=results.epoch
            )

    run_metrics.append((map_5095, results.save_dir, label, task.id))
    task.close()
    return map_5095

good_config = {
    "lr0": 0.01,
}
train_model(good_config, label="Final_Tune_2")


# 📦 Save best/worst models
run_metrics.sort(key=lambda x: x[0], reverse=True)
(best_score, best_dir, best_label, best_id), (worst_score, worst_dir, worst_label, worst_id) = run_metrics[0], run_metrics[-1]

save_root = Path("saved_models")
save_root.mkdir(exist_ok=True)

shutil.copy(best_dir / "weights/best.pt", save_root / "best.pt")
with open(save_root / "best_metrics.txt", "w") as f:
    f.write(f"Best mAP50-95: {best_score:.4f}\\nFrom: {best_dir.name} ({best_label})\\n")

shutil.copy(worst_dir / "weights/best.pt", save_root / "worst.pt")
with open(save_root / "worst_metrics.txt", "w") as f:
    f.write(f"Worst mAP50-95: {worst_score:.4f}\\nFrom: {worst_dir.name} ({worst_label})\\n")

# 🏷 Tag in ClearML
ClearMLTask.get_task(task_id=best_id).add_tags(["Best-Model"])
ClearMLTask.get_task(task_id=worst_id).add_tags(["Worst-Model"])
    
# 📊 Export to CSV
df = pd.DataFrame(run_metrics, columns=["mAP50-95", "run_dir", "label", "task_id"])
df.sort_values("mAP50-95", ascending=False).to_csv("saved_models/hpo_summary.csv", index=False)

print(f"\n✅ Best: {best_label} → {best_score:.4f}")
print(f"❌ Worst: {worst_label} → {worst_score:.4f}")
print(f"📊 Absolute Gain: {(best_score - worst_score):.4f}")
print(f"📈 Relative % Gain: {100 * (best_score - worst_score) / (worst_score + 1e-9):.2f}%")


New https://pypi.org/project/ultralytics/8.3.130 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.114  Python-3.9.18 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov9t.pt, data=balanced_dataset_split/train_80.yaml, epochs=50, time=None, patience=100, batch=180, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=clearml_yolo9_tuning_extended, name=yolo9t_Final_Tune_2_lr0.01, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=

train: Scanning C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\labels.cache... 29477 images, 2570 backgrounds, 0 corrupt: 100%|██████████| 29477/29477 [00:00<?, ?it/s]

train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478021875081281646_jpg.rf.bEZPhuyXU5hIovwQSTIp.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478021875081281646_jpg.rf.e9552980cf8c6fef4aa02cb84c6364f5.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478897760163798179_jpg.rf.5Pzrj3Eg3vZuyl7ztKAt.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478897760163798179_jpg.rf.98623be50b02ff17d58f89fddf7a0c6c.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Se

val: Fast image access  (ping: 0.20.1 ms, read: 76.413.9 MB/s, size: 36.7 KB)


val: Scanning C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset\valid\labels.cache... 2980 images, 350 backgrounds, 0 corrupt: 100%|██████████| 2980/2980 [00:00<?, ?it/s]


...

      50/50      4.63G      1.317     0.7041     0.9951        993        256: 100%|██████████| 164/164 [01:50<00:00,  1.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:21<00:00,  2.37s/it]

                   all       2980      19514      0.581      0.412      0.438      0.216



50 epochs completed in 1.966 hours.
Optimizer stripped from clearml_yolo9_tuning_extended\yolo9t_Final_Tune_2_lr0.01\weights\last.pt, 4.6MB
Optimizer stripped from clearml_yolo9_tuning_extended\yolo9t_Final_Tune_2_lr0.01\weights\best.pt, 4.6MB

Validating clearml_yolo9_tuning_extended\yolo9t_Final_Tune_2_lr0.01\weights\best.pt...
Ultralytics 8.3.114  Python-3.9.18 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv9t summary (fused): 197 layers, 1,972,929 parameters, 0 gradients, 7.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:21<00:00,  2.42s/it]


                   all       2980      19514      0.535      0.446      0.455      0.218
                   car        257        405       0.49      0.304      0.301      0.128
            pedestrian       2579      12667      0.715      0.717      0.748      0.444
      trafficLight-Red        727       2292      0.378      0.301      0.255     0.0927
    trafficLight-Green        296        489      0.456      0.407      0.434       0.22
                 truck        393       1043      0.588      0.358      0.347       0.13
          trafficLight         40         56      0.257      0.232      0.202     0.0962
                 biker        545       1426      0.691      0.466      0.504      0.224
  trafficLight-RedLeft        284        377      0.823      0.379      0.506      0.228
trafficLight-GreenLeft         34         65      0.343      0.338       0.27      0.116
   trafficLight-Yellow          5          5      0.459        0.8      0.795      0.312
trafficLight-YellowLe

In [ ]:
# ✅ Fetch hyperparameters for best and worst
best_hyperparams = None
worst_hyperparams = None

for t in ClearMLTask.get_tasks(task_ids=[best_id, worst_id]):
    if t.id == best_id:
        best_hyperparams = t.get_parameters()
    elif t.id == worst_id:
        worst_hyperparams = t.get_parameters()

print(f"\n✅ Best: {best_label} → {best_score:.4f}")
print(f"   ↳ Hyperparameters: {best_hyperparams}")
print(f"❌ Worst: {worst_label} → {worst_score:.4f}")
print(f"   ↳ Hyperparameters: {worst_hyperparams}")
print(f"📊 Absolute Gain: {(best_score - worst_score):.4f}")
print(f"📈 Relative % Gain: {100 * (best_score - worst_score) / (worst_score + 1e-9):.2f}%")


In [2]:
from ultralytics import YOLO
from clearml import Task
import optuna
import random
import shutil
from pathlib import Path
import pandas as pd
from clearml import Task as ClearMLTask
import os
import torch, gc


# 🟦 Main ClearML task to track the study
parent_task = Task.init(
    project_name="YOLOv9t Tuning Extended",
    task_name="YOLOv9t Local Optuna HPO",
    task_type=Task.TaskTypes.optimizer
)

dataset_yaml = "balanced_dataset_split/train_80.yaml"
results_path = Path("runs/detect")
run_metrics = []
save_root = Path("saved_models")
save_root.mkdir(exist_ok=True)

# 🔴 Add manual config run (moderately)
baseline_config = {
    "lr0": 0.01,           # same (still high, destabilizing early training)
    "momentum": 0.63,       # worse stability
    "weight_decay": 5e-3    ,  # stronger over-regularization
    "warmup_epochs": 0,    # no change, still bad
    "hsv_h": 0.2,          # more color distortion
    "mosaic": 0.1,         # less helpful spatial augmentation
    "mixup": 0.5,          # overly strong blending
    "degrees": 25,         # harsher rotation
    "scale": 0.3           # less scale variation (hurts generalization)
}

run_name = f"manual_bad_lr{baseline_config['lr0']:.0e}_mom{baseline_config['momentum']:.2f}"
task = Task.create(
    project_name="YOLOv9t Tuning Extended",
    task_name=f"YOLOv9t Run - {run_name}",
    task_type=Task.TaskTypes.training
)
task.connect(baseline_config)
task.add_tags(["YOLOv9t-Baseline", "Manual-Baseline"])
Task.current_task().close()
Task._Task__default_task = task
model = YOLO("yolov9t.pt")
results = model.train(
    optimizer="SGD",
    data=dataset_yaml,
    imgsz=256,
    epochs=50,
    batch=180,
    **baseline_config,
    project="clearml_yolo9_optuna",
    name=run_name,
    save=True,
    verbose=True,
)

try:
    map_5095 = results.box.map
    task.get_logger().report_scalar("metrics", "mAP50-95", map_5095, iteration=0)
except:
    map_5095 = 0.0
run_metrics.append((map_5095, results.save_dir, run_name, task.id))
task.close()
torch.cuda.empty_cache()
gc.collect()
del model

# 🔵 Add YOLO default config run (Ultralytics default hyperparameters)
default_run_name = "manual_default_yolov9t"
default_task = Task.create(
    project_name="YOLOv9t Tuning Extended",
    task_name=f"YOLOv9t Run - {default_run_name}",
    task_type=Task.TaskTypes.training
)
default_task.add_tags(["YOLOv9t-Baseline", "Manual-Default"])
Task.current_task().close()
Task._Task__default_task = default_task

default_model = YOLO("yolov9t.pt")
default_results = default_model.train(
    optimizer="SGD",
    data=dataset_yaml,
    imgsz=256,
    epochs=50,
    batch=180,
    project="clearml_yolo9_optuna",
    name=default_run_name,
    save=True,
    verbose=True,
)

try:
    default_map_5095 = default_results.box.map
    default_task.get_logger().report_scalar("metrics", "mAP50-95", default_map_5095, iteration=0)
except:
    default_map_5095 = 0.0

run_metrics.append((default_map_5095, default_results.save_dir, default_run_name, default_task.id))
default_task.close()
torch.cuda.empty_cache()
gc.collect()
del default_model


# 🎯 Training function to be optimized by Optuna
def objective(trial):
    hyperparameters = {
        "lr0": trial.suggest_float("lr0", 1e-5, 5e-3, log=True),          # fine-tune safe range
        "momentum": trial.suggest_float("momentum", 0.85, 0.97),          # wider range helps test stability
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True),
        "warmup_epochs": trial.suggest_int("warmup_epochs", 1, 5),        # 0 often unstable
        "hsv_h": trial.suggest_float("hsv_h", 0.0, 0.05),                 # excessive hue hurts color fidelity
        "mosaic": trial.suggest_float("mosaic", 0.5, 1.0),                # below 0.5 harms generalization
        "mixup": trial.suggest_float("mixup", 0.0, 0.3),                  # >0.3 makes blurry training
        "degrees": trial.suggest_int("degrees", 0, 10),                   # >10 leads to odd object angles
        "scale": trial.suggest_float("scale", 0.4, 1.0),                  # safe range for scale jitter
    }

    run_name = f"optuna_lr{hyperparameters['lr0']:.0e}_mom{hyperparameters['momentum']:.2f}"

    # ✅ Use Task.create instead of Task.init
    task = Task.create(
        project_name="YOLOv9t Tuning Extended",
        task_name=f"YOLOv9t Run - {run_name}",
        task_type=Task.TaskTypes.training
    )
    task.connect(hyperparameters)
    task.add_tags(["YOLOv9t-Optuna"])
    Task.current_task().close()
    Task._Task__default_task = task  # HACK: Reset default task context

    model = YOLO("yolov9t.pt")
    results = model.train(
        optimizer="SGD",
        data=dataset_yaml,
        imgsz=256,
        epochs=50,
        batch=180,
        **hyperparameters,
        project="clearml_yolo9_optuna",
        name=run_name,
        save=True,
        verbose=True,
    )
    try:
        map_5095 = results.box.map
        task.get_logger().report_scalar("metrics", "mAP50-95", map_5095, iteration=0)
    except:
        map_5095 = 0.0

    run_metrics.append((map_5095, results.save_dir, run_name, task.id))
    task.close()
    
    torch.cuda.empty_cache()
    gc.collect()
    del model
    return map_5095

# 🔁 Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=4)

# 📦 Save best/worst models
run_metrics.sort(key=lambda x: x[0], reverse=True)
(best_score, best_dir, best_label, best_id), (worst_score, worst_dir, worst_label, worst_id) = run_metrics[0], run_metrics[-1]

shutil.copy(best_dir / "weights/best.pt", save_root / "best.pt")
with open(save_root / "best_metrics.txt", "w") as f:
    f.write(f"Best mAP50-95: {best_score:.4f}\nFrom: {best_dir.name} ({best_label})\n")

shutil.copy(worst_dir / "weights/best.pt", save_root / "worst.pt")
with open(save_root / "worst_metrics.txt", "w") as f:
    f.write(f"Worst mAP50-95: {worst_score:.4f}\nFrom: {worst_dir.name} ({worst_label})\n")

ClearMLTask.get_task(task_id=best_id).add_tags(["Best-Model"])
ClearMLTask.get_task(task_id=worst_id).add_tags(["Worst-Model"])

# 📊 Export to CSV
df = pd.DataFrame(run_metrics, columns=["mAP50-95", "run_dir", "label", "task_id"])
df.sort_values("mAP50-95", ascending=False).to_csv("saved_models/optuna_hpo_summary.csv", index=False)

print(f"\n✅ Best: {best_label} → {best_score:.4f}")
print(f"❌ Worst: {worst_label} → {worst_score:.4f}")
print(f"📊 Absolute Gain: {(best_score - worst_score):.4f}")
print(f"📈 Relative % Gain: {100 * (best_score - worst_score) / (worst_score + 1e-9):.2f}%")


ClearML Task: created new task id=19f5dd626ecd42f49d5a6eaae65c4931
2025-05-09 04:53:25,013 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/6985ff7df0b841b8be48df830fa4faee/experiments/19f5dd626ecd42f49d5a6eaae65c4931/output/log
New https://pypi.org/project/ultralytics/8.3.129 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.114  Python-3.9.18 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov9t.pt, data=balanced_dataset_split/train_80.yaml, epochs=50, time=None, patience=100, batch=180, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=clearml_yolo9_optuna, name=manual_bad_lr1e-02_mom0.63, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, free

train: Scanning C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\labels.cache... 29477 images, 2570 backgrounds, 0 corrupt: 100%|██████████| 29477/29477 [00:00<?, ?it/s]

train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478021875081281646_jpg.rf.bEZPhuyXU5hIovwQSTIp.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478021875081281646_jpg.rf.e9552980cf8c6fef4aa02cb84c6364f5.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478897760163798179_jpg.rf.5Pzrj3Eg3vZuyl7ztKAt.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset_split\train_80\images\1478897760163798179_jpg.rf.98623be50b02ff17d58f89fddf7a0c6c.jpg: 1 duplicate labels removed
train: C:\Users\aariz\Documents\UMD\Semesters\Se

val: Fast image access  (ping: 0.10.0 ms, read: 171.365.0 MB/s, size: 36.7 KB)


val: Scanning C:\Users\aariz\Documents\UMD\Semesters\Semester 2\MSML 605\Project\Self Driving Car.v3-fixed-small.yolov9\balanced_dataset\valid\labels.cache... 2980 images, 350 backgrounds, 0 corrupt: 100%|██████████| 2980/2980 [00:00<?, ?it/s]


...

                   all       2980      19514      0.309      0.293      0.246      0.122

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      22/50      5.83G      1.505     0.9908      1.053       1487        256: 100%|██████████| 164/164 [01:56<00:00,  1.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:21<00:00,  2.40s/it]


                   all       2980      19514      0.307      0.269      0.244       0.12

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      23/50      5.59G      1.503     0.9868      1.052       1688        256:  45%|████▍     | 73/164 [00:51<01:04,  1.42it/s]
[W 2025-05-09 13:30:40,614] Trial 2 failed with parameters: {'lr0': 0.0008506455949248992, 'momentum': 0.8906522970614962, 'weight_decay': 8.01813909841436e-06, 'warmup_epochs': 4, 'hsv_h': 0.013764459881876523, 'mosaic': 0.7033895456449926, 'mixup': 0.024149101182643396, 'degrees': 7, 'scale': 0.6823569817783262} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\aariz\anaconda3\envs\tf\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\aariz\AppData\Local\Temp\ipykernel_34384\1214555369.py", line 139, in objective
    results = model.train(
  File "c:\Users\aariz\anaconda3\envs\tf\lib\site-packages\ultralytics\engine\model.py", line 790, in train
    self.trainer.train()
  File "c:\Users\aariz\anaconda3\envs\tf\lib\site-packages\ultraly

KeyboardInterrupt: 